# Lab 1: Building the Campus Event Management Agent

**Objective**: Build a production-ready AI agent using Microsoft Agents Framework

## What You'll Learn

- Define tools with type hints and auto-generate schemas
- Build agents with Microsoft Agents Framework
- Handle multi-turn conversations with threads
- Test agent functionality interactively

## Use Case: Campus Event Discovery & Registration

Your agent will help students with event-related queries and actions:

### Tools You'll Build:
1. **get_events()** - Browse all available campus events (READ)
2. **get_event_details()** - Get detailed info about a specific event (READ)
3. **register_for_event()** - Sign up for an event (WRITE)
4. **get_event_participants()** - See who's registered for an event (READ)

**Key Feature**: Mix of read (GET) and write (POST) operations - a realistic agent!

### After This Lab:
You can extend this pattern to build tools for:
- 📍 **Venues** - Check availability, book spaces (see mock backend endpoints)
- 📢 **Notifications** - Send announcements to participants

---

## Part A: Setup

In [6]:
!pip install -q agent-framework --pre requests fastapi uvicorn pyngrok nest-asyncio

In [7]:
# Import required libraries
from google.colab import userdata
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient
from typing import Annotated
from pydantic import Field
import requests
import sys
sys.path.append('/content')  # For utils.py in Colab

print("✅ Libraries imported")

✅ Libraries imported


**Provide Notebook Access to the Secrets**

- Look at the left sidebar of this Colab notebook
- Click the 🔑 key icon (Secrets)
- Toggle "Notebook access" to ON for `GITHUB_PAT` secret

In [25]:
# Load configuration from Lab 0
GITHUB_PAT = userdata.get('GITHUB_PAT')
BACKEND_URL = "https://51c4e6e4c316.ngrok-free.app/"  # TODO: Paste your ngrok URL from Lab 0

if not BACKEND_URL:
    print("⚠️ WARNING: Set BACKEND_URL from Lab 0!")
else:
    print(f"✅ Configuration loaded")
    print(f"   Backend: {BACKEND_URL}")

✅ Configuration loaded
   Backend: https://51c4e6e4c316.ngrok-free.app/


In [26]:
# Download utils.py helper functions
!wget -q https://raw.githubusercontent.com/tezansahu/building-eval-driven-ai-agents/main/labs/utils.py -O utils.py

# Or for this workshop, create it inline
from utils import function_to_tool_schema, print_agent_response

print("✅ Utility functions loaded")

✅ Utility functions loaded


## Part B: Define Tool Functions

Tools are Python functions that the agent can call to perform actions.

### We'll Build 4 Event-Related Tools:
- 2 **READ tools** (GET requests) - Browse and discover
- 2 **WRITE tools** (POST requests) - Register and query participants

### Key Points:
- Use **type hints** (`Annotated[type, Field(description="...")]`) for auto-schema generation
- Write clear **docstrings** - the LLM reads these!
- Return **informative strings** - the LLM uses these to craft responses
- Handle **errors gracefully** with try/except

Let's looks at the first READ tool (pre-implemented):

In [27]:
# Tool 1: Get All Events (READ)
def get_events() -> str:
    """
    Retrieve a list of all available campus events.

    Use this when users ask about things like upcoming events happening, that they could attend.

    Returns a formatted list of events with names, dates, and venues.
    """
    try:
        response = requests.get(f"{BACKEND_URL}/events")
        events = response.json()

        if not events:
            return "No events are currently available."

        # Format event list
        result = f"Found {len(events)} events:\n\n"
        for event in events:
            result += f"• {event['name']} (ID: {event['event_id']})\n"
            result += f"  📅 {event['date']} at {event['time']}\n"
            result += f"  📍 {event['venue']}\n"
            result += f"  👥 {len(event.get('participants', []))}/{event['max_participants']} registered\n\n"

        return result.strip()
    except Exception as e:
        return f"Error fetching events: {str(e)}"

print("✅ Tool 1 defined: get_events()")

✅ Tool 1 defined: get_events()


Now, let's try to implement the 2nd READ tool...

In [28]:
# Tool 2: Get Event Details (READ)
def get_event_details(
    event_id: Annotated[str, Field(description="Event identifier")]
) -> str:
    """
    Get detailed information about a specific event.

    Use this when users want more info about a particular event:
    - "Tell me about TechFest"
    - "What's the AI Workshop about?"
    - "Details on the hackathon"

    Returns event description, schedule, venue, and capacity info.
    """

    try:
        response = requests.get(f"{BACKEND_URL}/events/{event_id}")

        if response.status_code == 404:
            return f"Event '{event_id}' not found. Use get_events() to see available events."

        event = response.json()

        result = f"📋 {event['name']}\n\n"
        result += f"Description: {event['description']}\n"
        result += f"📅 When: {event['date']} at {event['time']}\n"
        result += f"📍 Where: {event['venue']}\n"
        result += f"👥 Capacity: {len(event.get('participants', []))}/{event['max_participants']}\n"

        if len(event.get('participants', [])) >= event['max_participants']:
            result += "\n⚠️ Event is FULL"
        else:
            result += f"\n✅ {event['max_participants'] - len(event.get('participants', []))} spots available"

        return result
    except Exception as e:
        return f"Error fetching event details: {str(e)}"

<details>
<summary><b>Solution</b></summary>
  
```python
def get_event_details(
    event_id: Annotated[str, Field(description="Event identifier")]
) -> str:
    """
    Get detailed information about a specific event.
    
    Use this when users want more info about a particular event:
    - "Tell me about TechFest"
    - "What's the AI Workshop about?"
    - "Details on the hackathon"
    
    Returns event description, schedule, venue, and capacity info.
    """
    
    try:
        response = requests.get(f"{BACKEND_URL}/events/{event_id}")
        
        if response.status_code == 404:
            return f"Event '{event_id}' not found. Use get_events() to see available events."
        
        event = response.json()
        
        result = f"📋 {event['name']}\n\n"
        result += f"Description: {event['description']}\n"
        result += f"📅 When: {event['date']} at {event['time']}\n"
        result += f"📍 Where: {event['venue']}\n"
        result += f"👥 Capacity: {len(event.get('participants', []))}/{event['max_participants']}\n"
        
        if len(event.get('participants', [])) >= event['max_participants']:
            result += "\n⚠️ Event is FULL"
        else:
            result += f"\n✅ {event['max_participants'] - len(event.get('participants', []))} spots available"
        
        return result
    except Exception as e:
        return f"Error fetching event details: {str(e)}"
```
</details>

Now, let's try to implement our first WRITE tool...

In [29]:
# Tool 3: Register for Event (WRITE)
def register_for_event(
    student_id: Annotated[str, Field(description="Unique student ID")],
    event_id: Annotated[str, Field(description="Event identifier")],
    student_name: Annotated[str, Field(description="Student's full name")]
) -> str:
    """
    Register a student for a campus event.

    Use this when a student wants to sign up or register for an event.
    Returns confirmation with event details.
    """

    try:
        response = requests.post(
            f"{BACKEND_URL}/events/{event_id}/register",
            json={"student_id": student_id, "student_name": student_name}
        )
        result = response.json()

        if result.get("success"):
            details = result.get("event_details", {})
            confirmation = f"✅ Successfully registered {student_name} for {details.get('event_name', 'event')}!\n\n"
            confirmation += f"📅 Date: {details.get('date', 'TBD')}\n"
            confirmation += f"⏰ Time: {details.get('time', 'TBD')}\n"
            confirmation += f"📍 Venue: {details.get('venue', 'TBD')}\n"
            confirmation += f"👥 Registered participants: {details.get('participants_count', '?')}"
            return confirmation
        else:
            return f"❌ Registration failed: {result.get('message')}"
    except Exception as e:
        return f"Error during registration: {str(e)}"

<details>
<summary><b>Solution</b></summary>
  
```python
def register_for_event(
    student_id: Annotated[str, Field(description="Unique student ID")],
    event_id: Annotated[str, Field(description="Event identifier")],
    student_name: Annotated[str, Field(description="Student's full name")]
) -> str:
    """
    Register a student for a campus event.
    
    Use this when a student wants to sign up or register for an event.
    Returns confirmation with event details.
    """
    
    try:
        response = requests.post(
            f"{BACKEND_URL}/events/{event_id}/register",
            json={"student_id": student_id, "student_name": student_name}
        )
        result = response.json()
        
        if result.get("success"):
            details = result.get("event_details", {})
            confirmation = f"✅ Successfully registered {student_name} for {details.get('event_name', 'event')}!\n\n"
            confirmation += f"📅 Date: {details.get('date', 'TBD')}\n"
            confirmation += f"⏰ Time: {details.get('time', 'TBD')}\n"
            confirmation += f"📍 Venue: {details.get('venue', 'TBD')}\n"
            confirmation += f"👥 Registered participants: {details.get('participants_count', '?')}"
            return confirmation
        else:
            return f"❌ Registration failed: {result.get('message')}"
    except Exception as e:
        return f"Error during registration: {str(e)}"
```
</details>

Here's our last tool:

In [30]:
# Tool 4: Get Event Participants (READ)
def get_event_participants(
    event_id: Annotated[str, Field(description="Event identifier")]
) -> str:
    """
    Get the list of students registered for an event.

    Use this when users ask about participants, or people registered for an event.

    Returns participant count and list of registered students.
    """
    try:
        response = requests.get(f"{BACKEND_URL}/events/{event_id}/participants")

        if response.status_code == 404:
            return f"Event '{event_id}' not found."

        data = response.json()

        event_name = data.get('event_name', 'Event')
        participant_count = data.get('participant_count', 0)
        participants = data.get('participants', [])

        if participant_count == 0:
            return f"No one has registered for {event_name} yet."

        result = f"📊 {event_name} Registrations\n\n"
        result += f"Total participants: {participant_count}\n\n"
        result += "Registered students:\n"
        for i, student_id in enumerate(participants, 1):
            result += f"{i}. {student_id}\n"

        return result
    except Exception as e:
        return f"Error fetching participants: {str(e)}"

print("✅ Tool 4 defined: get_event_participants()")

✅ Tool 4 defined: get_event_participants()


## Part C: Auto-Generate Tool Schemas

**Why auto-generate?**
- No manual schema writing (error-prone!)
- Type hints ensure consistency
- Single source of truth (the function itself)

In [31]:
# Auto-generate tool schemas using utility function
# No manual schema writing needed!

# Note: With agent-framework, we can pass functions directly
# The framework handles schema generation internally
# But let's verify our schemas are correct:

from utils import function_to_tool_schema
import json

In [32]:
# Test schema generation for a tool with NO parameters (get_events)
print("Schema for get_events() - NO parameters:")
print(json.dumps(function_to_tool_schema(get_events), indent=2))

Schema for get_events() - NO parameters:
{
  "type": "function",
  "function": {
    "name": "get_events",
    "description": "Retrieve a list of all available campus events.\n\nUse this when users ask about things like upcoming events happening, that they could attend.\n\nReturns a formatted list of events with names, dates, and venues.",
    "parameters": {
      "type": "object",
      "properties": {},
      "required": []
    }
  }
}


In [33]:
# Test schema generation for a tool WITH parameters (register_for_event)
print("Schema for register_for_event() - WITH parameters:")
print(json.dumps(function_to_tool_schema(register_for_event), indent=2))

Schema for register_for_event() - WITH parameters:
{
  "type": "function",
  "function": {
    "name": "register_for_event",
    "description": "Register a student for a campus event.\n\nUse this when a student wants to sign up or register for an event.\nReturns confirmation with event details.",
    "parameters": {
      "type": "object",
      "properties": {
        "student_id": {
          "type": "string",
          "description": "Unique student ID"
        },
        "event_id": {
          "type": "string",
          "description": "Event identifier"
        },
        "student_name": {
          "type": "string",
          "description": "Student's full name"
        }
      },
      "required": [
        "student_id",
        "event_id",
        "student_name"
      ]
    }
  }
}


## Part D: Create the Agent

### Agent Components:
1. **Chat Client** - Connects to LLM (GitHub Models)
2. **Instructions** - System prompt that guides agent behavior
3. **Tools** - Functions the agent can call

### Why Microsoft Agents Framework?
- ✅ **Automatic orchestration** - No manual message loops
- ✅ **Built-in function calling** - Handles tool execution automatically
- ✅ **Thread management** - Multi-turn conversations made easy
- ✅ **Error handling** - Graceful fallbacks for tool errors

In [34]:
# Initialize chat client
chat_client = OpenAIChatClient(
    model_id="gpt-4o-mini",
    api_key=GITHUB_PAT,
    base_url="https://models.github.ai/inference"
)

print("✅ Chat client initialized")

✅ Chat client initialized


In [35]:
# Define agent instructions
# TODO: Complete the instructions with additional guidelines

AGENT_INSTRUCTIONS = """You are a helpful campus event management assistant at an engineering college.

Your capabilities:
- Register students for campus events using register_for_event()
- Book venues for clubs and organizations using book_venue()
- Send notifications to event participants using send_event_notification()

Guidelines:
- Be friendly and concise (under 50 words)
- ALWAYS include specific details from tool results (event names, dates, venues)
- If information is missing, politely ask for it before calling tools
- Confirm successful actions with details"""


print("✅ Agent instructions defined")

✅ Agent instructions defined


<details>
<summary><b>Solution</b></summary>
  
```md
You are a helpful campus event management assistant at an engineering college.

Your capabilities:
- Register students for campus events using register_for_event()
- Book venues for clubs and organizations using book_venue()
- Send notifications to event participants using send_event_notification()

Guidelines:
- Be friendly and concise (under 50 words)
- ALWAYS include specific details from tool results (event names, dates, venues)
- If information is missing, politely ask for it before calling tools
- Confirm successful actions with details
```
</details>

In [36]:
# Create the agent with all tools
campus_agent = ChatAgent(
    chat_client=chat_client,
    instructions=AGENT_INSTRUCTIONS,
    tools=[
        get_events,              # READ - Browse all events
        get_event_details,       # READ - Specific event info
        register_for_event,      # WRITE - Sign up for event
        get_event_participants   # READ - Who's registered
    ]
)

print("✅ Campus Event Agent created successfully!")
print(f"   Tools: 4")
print(f"   Model: gpt-4o-mini")

✅ Campus Event Agent created successfully!
   Tools: 4
   Model: gpt-4o-mini


## Part E: Test Individual Tools

In [37]:
# Test 1: Browse Events (READ)
response = await campus_agent.run(
    "What events are happening on campus?"
)

print_agent_response(response, show_details=True)

AGENT RESPONSE

Here are the upcoming events on campus:

1. **TechFest 2024**  
   📅 March 15, 2024 | 10:00-17:00  
   📍 Main Auditorium  

2. **Spring Hackathon 2024**  
   📅 April 20, 2024 | 09:00-21:00  
   📍 Computer Lab 1  

3. **AI & Machine Learning Workshop**  
   📅 March 25, 2024 | 14:00-17:00  
   📍 Seminar Hall B  

4. **Robotics Club Demo Day**  
   📅 April 5, 2024 | 15:00-18:00  
   📍 Engineering Workshop  

Let me know if you want to register for any of these events!


------------------------------------------------------------
DETAILS
------------------------------------------------------------

🔧 Tool Calls Made: 1
  1. get_events()


In [38]:
# Test 2: Event Details (READ)
response = await campus_agent.run(
    "Tell me more about the AI Workshop"
)

print_agent_response(response, show_details=True)

AGENT RESPONSE

The **AI & Machine Learning Workshop** is a hands-on session focused on building AI applications. It will take place on **March 25, 2024**, from **2:00 PM to 5:00 PM** at **Seminar Hall B**. There are **50 spots available**!


------------------------------------------------------------
DETAILS
------------------------------------------------------------

🔧 Tool Calls Made: 2
  1. get_events()
  2. get_event_details()
     - event_id: ai_workshop


In [39]:
# Test 3: Register for Event (WRITE)
response = await campus_agent.run(
    "I'm Priya with student ID S001. I want to register for the AI Workshop."
)

print_agent_response(response, show_details=True)

AGENT RESPONSE

You're successfully registered for the **AI & Machine Learning Workshop**! 

📅 Date: 2024-03-25  
🕒 Time: 14:00-17:00  
📍 Venue: Seminar Hall B  

Excited to see you there!


------------------------------------------------------------
DETAILS
------------------------------------------------------------

🔧 Tool Calls Made: 2
  1. get_events()
  2. register_for_event()
     - event_id: ai_workshop
     - student_id: S001
     - student_name: Priya


In [40]:
# Test 4: Get Participants (READ)
response = await campus_agent.run(
    "Who's registered for the AI Workshop?"
)

print_agent_response(response, show_details=True)

AGENT RESPONSE

The AI & Machine Learning Workshop on March 25, 2024, has 1 participant registered: Student ID S001. If you need more details, feel free to ask!


------------------------------------------------------------
DETAILS
------------------------------------------------------------

🔧 Tool Calls Made: 2
  1. get_events()
  2. get_event_participants()
     - event_id: ai_workshop


## Part F: Multi-Turn Conversations with Threads

**Threads enable:**
- Persistent conversation history
- Context retention across multiple queries
- Natural back-and-forth dialogue
- Serialization for storage/retrieval

In [41]:
# Create a new thread for a conversation
thread = campus_agent.get_new_thread()

print("✅ New conversation thread created")

✅ New conversation thread created


Here's an example of a multi-turn conversation, where context is maintained across multiple turns.

In [42]:
# Turn 1: User introduces themselves
response1 = await campus_agent.run(
    "Hi! My name is Rahul and my student ID is S002.",
    thread=thread
)
print("Turn 1 (Introduction):")
print_agent_response(response1, show_details=True)

Turn 1 (Introduction):
AGENT RESPONSE

Hi Rahul! How can I assist you today? Would you like to register for an event, or do you need information about something specific?


------------------------------------------------------------
DETAILS
------------------------------------------------------------

🔧 Tool Calls Made: None


In [43]:
# Turn 2: User asks about events (browsing)
response2 = await campus_agent.run(
    "What events can I attend?",
    thread=thread
)
print("Turn 2 (Browse Events):")
print_agent_response(response2, show_details=True)

Turn 2 (Browse Events):
AGENT RESPONSE

Here are the upcoming events you can attend:

1. **TechFest 2024**  
   📅 March 15, 2024 | ⏰ 10:00-17:00 | 📍 Main Auditorium

2. **Spring Hackathon 2024**  
   📅 April 20, 2024 | ⏰ 09:00-21:00 | 📍 Computer Lab 1

3. **AI & Machine Learning Workshop**  
   📅 March 25, 2024 | ⏰ 14:00-17:00 | 📍 Seminar Hall B

4. **Robotics Club Demo Day**  
   📅 April 5, 2024 | ⏰ 15:00-18:00 | 📍 Engineering Workshop

Let me know if you’d like to register for any of these events!


------------------------------------------------------------
DETAILS
------------------------------------------------------------

🔧 Tool Calls Made: 1
  1. get_events()


In [44]:
# Turn 3: User asks for details (should use event from previous turn)
response3 = await campus_agent.run(
    "Tell me more about TechFest 2024",
    thread=thread
)
print("Turn 3 (Event Details):")
print_agent_response(response3, show_details=True)

Turn 3 (Event Details):
AGENT RESPONSE

**TechFest 2024** is an annual technical festival featuring coding competitions, robotics, and tech talks. 

- 📅 Date: March 15, 2024  
- ⏰ Time: 10:00-17:00  
- 📍 Venue: Main Auditorium  
- 👥 Capacity: 500 spots available 

Would you like to register for it?


------------------------------------------------------------
DETAILS
------------------------------------------------------------

🔧 Tool Calls Made: 1
  1. get_event_details()
     - event_id: techfest2024


> **Notice:** The agent doesn't need to call the `get_events()` tool before the `get_event_details()` tool, because if already has that information from the previous turn.

In [45]:
# Turn 4: User registers (agent should use remembered ID and name)
response4 = await campus_agent.run(
    "Register me for this.",
    thread=thread
)
print("Turn 4 (Registration):")
print_agent_response(response4, show_details=True)

Turn 4 (Registration):
AGENT RESPONSE

You're successfully registered for **TechFest 2024**! 

- 📅 Date: March 15, 2024  
- ⏰ Time: 10:00-17:00  
- 📍 Venue: Main Auditorium

Excited to see you there! If you need anything else, just let me know!


------------------------------------------------------------
DETAILS
------------------------------------------------------------

🔧 Tool Calls Made: 1
  1. register_for_event()
     - event_id: techfest2024
     - student_id: S002
     - student_name: Rahul


> **Notice:** Agent remembered Rahul's name and ID from 1st turn, and also understood that "this" here means "Tech Fest 2024".

## Part G: Interactive Chat Session

Try your agent in an interactive mode!

In [46]:
# Interactive chat loop
# Type 'quit' or 'exit' to end the conversation

async def interactive_chat():
    """Run interactive chat session with the agent."""
    thread = campus_agent.get_new_thread()

    print("="*60)
    print("CAMPUS EVENT AGENT - Interactive Chat")
    print("="*60)
    print("Type your message (or 'quit' to exit)\n")

    while True:
        # Get user input
        user_input = input("You: ").strip()

        if user_input.lower() in ['quit', 'exit', 'q']:
            print("\nGoodbye! 👋")
            break

        if not user_input:
            continue

        # Get agent response
        response = await campus_agent.run(user_input, thread=thread)

        print(f"\nAgent: {response.text}\n")
        print("-"*60)


In [47]:
# Run interactive chat
await interactive_chat()

CAMPUS EVENT AGENT - Interactive Chat
Type your message (or 'quit' to exit)

You: hi

Agent: Hello! How can I assist you today?

------------------------------------------------------------
You: register me at 1st event

Agent: I'll need your student ID and name to register you for the first event. Could you please provide those details?

------------------------------------------------------------
You: 123456

Agent: I still need your full name to complete the registration. Could you please share that?

------------------------------------------------------------
You: luxxx

Agent: It seems that the registration didn't go through. Let me check the details of the first event to assist you better. One moment, please!The first event is **TechFest 2024** on **March 15, 2024**, from **10:00 to 17:00** at the **Main Auditorium**. 

I'll try registering you for this event again with the details you provided. One moment!You're successfully registered for **TechFest 2024** on **March 15, 2024*

## Part H: Test Complex Scenarios

In [48]:
# Scenario 1: Discovery → Details → Register (multi-step)
response = await campus_agent.run(
    "Show me the events, then give me details about the AI Workshop, and register me as Amit (S003)."
)

print("Scenario 1: Multi-step task (browse → details → register)")
print_agent_response(response, show_details=True)

Scenario 1: Multi-step task (browse → details → register)
AGENT RESPONSE

Here are the events:

1. **TechFest 2024**: March 15, 2024, at Main Auditorium
2. **Spring Hackathon 2024**: April 20, 2024, at Computer Lab 1
3. **AI & Machine Learning Workshop**: March 25, 2024, at Seminar Hall B
4. **Robotics Club Demo Day**: April 5, 2024, at Engineering Workshop

Details for the **AI Workshop**:
- **Description**: Hands-on workshop on building AI applications
- **Date**: March 25, 2024
- **Time**: 14:00-17:00
- **Venue**: Seminar Hall B
- **Capacity**: 1/50 (49 spots available)

You are successfully registered for the AI & Machine Learning Workshop, Amit! 🎉


------------------------------------------------------------
DETAILS
------------------------------------------------------------

🔧 Tool Calls Made: 3
  1. get_events()
  2. get_event_details()
     - event_id: ai_workshop
  3. register_for_event()
     - event_id: ai_workshop
     - student_id: S003
     - student_name: Amit


In [49]:
# Scenario 2: Missing information (agent should ask)
response = await campus_agent.run(
    "I want to register for an event."
)

print("Scenario 2: Missing information")
print(f"Agent: {response.text}")
print("\n💡 Notice: Agent asks for missing details instead of guessing!")

Scenario 2: Missing information
Agent: Could you please provide me with the name of the event you'd like to register for, along with your student ID and full name?

💡 Notice: Agent asks for missing details instead of guessing!


In [50]:
# Scenario 3: General query (no tool needed)
response = await campus_agent.run(
    "What can you help me with regarding campus events?"
)

print("Scenario 3: Information query")
print(f"Agent: {response.text}")
print("\n💡 Notice: Agent responds without calling any tools when appropriate!")

Scenario 3: Information query
Agent: I can assist you with registering for campus events, booking venues for clubs and organizations, and sending notifications to event participants. Let me know what you need help with!

💡 Notice: Agent responds without calling any tools when appropriate!


## 🎉 Lab 1 Complete!

### What You Accomplished:

✅ **Built 4 event-focused tools** - 3 READ (GET) + 1 WRITE (POST)  
✅ **Auto-generated schemas** using type hints (no manual writing!)  
✅ **Created an agent** with Microsoft Agents Framework  
✅ **Tested both reads and writes** - Browse → Details → Register flow  
✅ **Implemented multi-turn conversations** using threads  
✅ **Handled edge cases** (missing info, multi-step tasks)

### Key Learnings:

1. **Type hints are powerful** - They enable auto-schema generation
2. **Mix of READ and WRITE** - Real agents do both discovery and actions
3. **Agents Framework simplifies** - No manual loops or message handling
4. **Threads enable context** - Agents remember conversation history
5. **Descriptive docstrings matter** - The LLM reads them to decide which tool to use
6. **Tool responses are key** - Detailed returns help the LLM craft better responses

### What's Next?

**Lab 2**: Evaluate and improve this agent!
- Measure relevance and task adherence
- Identify issues systematically
- Improve based on metrics
- Quantify improvement

---

## 🚀 Extend Your Agent (Optional Challenge)

Now that you've mastered event management, apply the same pattern to build tools for:

### 1️⃣ Venue Management Tools
Explore the mock backend endpoints:
```python
# READ operations
GET /venues              # List all venues
GET /venues/{venue_id}   # Get venue details
GET /venues/{venue_id}/availability?date=YYYY-MM-DD  # Check availability

# WRITE operations
POST /venues/{venue_id}/book  # Book a venue
```

**Suggested tools:**
- `get_venues()` - List available venues
- `check_venue_availability()` - Check if venue is free
- `book_venue()` - Reserve a space

### 2️⃣ Notification Tools
```python
# WRITE operations
POST /notifications/send  # Send notifications
GET /notifications/log    # View notification history
```

**Suggested tools:**
- `send_event_notification()` - Notify participants
- `get_notification_history()` - View sent messages

### 3️⃣ Combined Multi-Domain Agent
Build a **super-agent** that handles:
- Events (already done! ✅)
- Venues (your extension)
- Notifications (your extension)

**Update instructions to:**
```python
SUPER_AGENT_INSTRUCTIONS = """
You are a comprehensive campus management assistant.

You can help with:
1. Events - Browse, get details, register, check participants
2. Venues - List spaces, check availability, book venues
3. Notifications - Send announcements to event participants

Choose the appropriate tool based on what the user needs.
"""
```

### Try It Yourself!

1. Pick one domain (Venues or Notifications)
2. Define 2-3 tools following the pattern from Part B
3. Update agent instructions
4. Test with queries like:
   - "Show me available venues"
   - "Book Seminar Hall B for tomorrow"
   - "Send a reminder to TechFest participants"

---

## Bonus: Experiment Further!

Try these challenges:

1. **Add error handling** - What happens if the backend is down?
2. **Add input validation** - Check date formats, student ID patterns
3. **Add unregister tool** - `DELETE /events/{event_id}/register/{student_id}`
4. **Improve instructions** - Make the agent more specific about when to use each tool
5. **Test edge cases** - Duplicate registrations, invalid event IDs, full events

### Architecture Benefits:

**vs. Raw OpenAI Function Calling:**
- ❌ Manual message loop → ✅ Automatic orchestration
- ❌ Manual schema writing → ✅ Auto-generated from type hints
- ❌ Manual thread management → ✅ Built-in thread support
- ❌ Manual error handling → ✅ Graceful fallbacks

**Production Ready:**
- Serializable threads for storage
- Async/await for scalability
- Type safety with Pydantic
- Clean separation of concerns